In [1]:
import numpy as np

In [15]:
[loss_list_global[i][0][:min_batch] for i in range(10)]

TypeError: 'int' object is not subscriptable

In [14]:
[1,2,3,4])

4

In [12]:
np.empty(shape=(1,1))

array([[6.91790308e-310]])

In [ ]:
        meta_optimizer = torch.optim.Adam(meta_net.parameters(), lr=1e-4, weight_decay=0.01)
    if  epoch > 0 :
                pseudo_net = ModelFedCon(args.model, args.out_dim, n_classes, args.net_config).cuda()
                pseudo_net.load_state_dict(net.state_dict())
                #pseudo_net = copy.deepcopy(net)
                #init_params(pseudo_net)
                pseudo_net.train()

                _,_,pseudo_outputs = pseudo_net(x)
                #print(pseudo_outputs)
                pseudo_loss_vector = functional.cross_entropy(pseudo_outputs, target.long(), reduction='none')
                pseudo_loss_vector_reshape = torch.reshape(pseudo_loss_vector, (-1, 1))
                pseudo_weight = meta_net(pseudo_loss_vector_reshape.data)
                pseudo_loss = torch.mean(pseudo_weight * pseudo_loss_vector_reshape)

                pseudo_grads = torch.autograd.grad(pseudo_loss, pseudo_net.parameters(), create_graph=True)

                pseudo_optimizer = MetaSGD(pseudo_net, pseudo_net.parameters(), lr=lr)
                pseudo_optimizer.load_state_dict(optimizer.state_dict())
                pseudo_optimizer.meta_step(pseudo_grads)

                del pseudo_grads

                try:
                    meta_inputs, meta_labels = next(meta_dataloader_iter)
                except StopIteration:
                    meta_dataloader_iter = iter(meta_dataloader)
                    meta_inputs, meta_labels = next(meta_dataloader_iter)

                meta_inputs, meta_labels = meta_inputs.to(args.device), meta_labels.to(args.device)
                _,_,meta_outputs = pseudo_net(meta_inputs)
                meta_loss = criterion(meta_outputs, meta_labels.long())

                meta_optimizer.zero_grad()
                meta_loss.backward()
                meta_optimizer.step()